In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import re

pd.set_option('display.max_rows', 10000000)
#Importing the relevant libraries for the project

In [ ]:
def path_finder(date):
    #this function will find the day of the week according to the day year and month
    #the purpose is to be able to find what day has more activity.
    day_name = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo']
    date = str(date)
    day = datetime.datetime.strptime(date, '%Y-%m-%d').weekday()
    day_found = day_name[day]
    return day_found


def twitter_cleaner(mes):
    #this function will clean completely the dataset given by twitter when you download by tweet.
    dataset = pd.read_csv('{}.csv'.format(mes)) #Reading the file
    mes = str(mes)
    dataset = pd.DataFrame(dataset) #Making sure the dataset is on a pandas DataFrame object type
    dataset['hora']=pd.to_datetime(dataset['hora']) # Changing feature content type
    dataset.drop(['ID del Tweet', 'impresiones promocionado',
 'interacciones promocionado',
 'tasa de interacción promocionado',
 'retweets promocionado',
 'respuestas promocionado',
 'me gusta promocionado',
 'clics de perfil de usuario promocionado',
 'clics en URL promocionado',
 'clics de etiquetas promocionado',
 'ampliaciones de detalles promocionado',
 'clics en enlaces permanentes promocionado',
 'Se abre la aplicación promocionado',
 'descargas de app promocionado',
 'seguimientos promocionado',
 'enviar Tweet por correo electrónico promocionado',
 'marcar teléfono promocionado',
 'visualizaciones multimedia promocionado',
 'interacciones con el contenido multimedia promocionado',
 'Se abre la aplicación', 'descargas de app',
 'enviar Tweet por correo electrónico', 'marcar teléfono', 'seguimientos', 'clics en enlaces permanentes'], axis=1, inplace=True) # Dropping not relevant columns
 
    dataset.columns = ['Enlace permanente de Tweet','Texto del Tweet', 'Hora', 'impresiones', 'interacciones',
       'tasa de interacción en %', 'retweets', 'respuestas', 'me gusta',
       'clics de perfil de usuario', 'clics en URL', 'clics de etiquetas',
       'ampliaciones de detalles', 'visualizaciones multimedia',
       'interacciones con el contenido multimedia']#changing the name of some columns
    dataset['tasa de interacción en %'] =  dataset['tasa de interacción en %'].apply(lambda x: x*100) #Engineering the columns so we can see it on percentage
    dataset['tasa de interacción en %'] = np.round(dataset['tasa de interacción en %'],2) #A simple rounding, too many decimals.

    list_to_integer = [
              'impresiones', 'interacciones','retweets', 'respuestas', 'me gusta',
       'clics de perfil de usuario', 'clics en URL', 'clics de etiquetas',
       'ampliaciones de detalles'] #Im too lazy to change those columns to integer one by one
    for name in list_to_integer:
        dataset[name] = np.int64(dataset[name])#changing the column to integer

    dataset['Fecha'] = dataset['Hora'].apply(str).copy()
    
    for i in range(0, int(dataset['Fecha'].shape[0])):
        dataset['Fecha'][i] = dataset['Fecha'][i][:10] #Fechas means date, and the actual kind of date I want is only with Year, month and day, not with those weird numbers it had.

    dataset['Dia'] = dataset['Fecha'].copy()
    dataset['Dia'] = dataset['Dia'].apply(lambda x: path_finder(x))#Implementing the path finder function previously described

    dataset['Fecha'] =pd.to_datetime(dataset['Fecha'])
    
    #dataset['Fecha'] = dataset['Fecha'].apply(pd.to_datetime)
    #dataset['hora'] = pd.to_datetime(dataset['hora'], format='%H%M')
    dataset['Hora'] = dataset['Hora'].apply(str) #Changing hora feature to string it was datetime
    e = list(dataset['Hora'].values)
    for i in range(0, len(e)):#This for will engineer the hora feature so the values will be two integers like 14, 14 hours of course.
        e[i] = e[i][11:13] 
    dataset['Hora'] = e
    dataset['Hora'] = np.int64(dataset['Hora'])
    #dataset['Hora'] = pd.Timestamp(dataset['Hora'], unit='h')
    dataset = dataset[['Texto del Tweet',
        'Fecha',
        'Dia',
        'Hora',
        'impresiones',
        'interacciones',
        'tasa de interacción en %',
        'retweets',
        'respuestas',
        'me gusta',
        'clics de perfil de usuario',
        'clics en URL',
        'clics de etiquetas',
        'ampliaciones de detalles',
        'visualizaciones multimedia',
        'interacciones con el contenido multimedia']] #making sure the features I want will be in the final result file
     
    dataset.to_csv('{}_clean.csv'.format(mes), encoding='utf-8', index=False) #Saving the engineering.
    round(dataset.describe(),2).to_csv('{}_described_clean.csv'.format(mes), encoding='utf-8', index=True) #A quick statistical description for the numeric values and saving it.

def fb_cleaner_alcance(name): #Short function to engineer the date column for facebook content data
    df = pd.read_csv('{}.csv'.format(name), encoding='utf-8')
    df['Fecha'] = df['Fecha'].apply(lambda x: x[0:10])
    df.to_csv('{}_clean.csv'.format(name), encoding='utf-8', index=False)


def fb_cleaner_contenido(name): #Short function to drop not relevant columns and engineering the hour columns, and saving it immediately
    df = pd.read_csv('{}.csv'.format(name), encoding='utf-8')
    try:
        df = df.drop(columns=['Resultados','Costo por resultado'])
    except Exception:
        pass
    df['Hora de publicación'] = df['Hora de publicación'].astype(str)
    df['Hora de publicación'] = df['Hora de publicación'].apply(lambda x: x[0:10])
    df['Dia'] = df['Hora de publicación'].copy()
    df['Dia'] = df['Dia'].apply(lambda x: path_finder(x))

    df.to_csv('{}_clean.csv'.format(name), encoding='utf-8', index=False)
    round(df.describe(),2).to_csv('{}_described.csv'.format(name))

def tiktok_overall(name):
    df = pd.read_csv('{}.csv'.format(name), encoding='utf-8')
    df['Dia'] = df['Fecha'].copy()
    df['Dia'] = df['Dia'].apply(lambda x: path_finder(x))
    df.to_csv('{}_clean.csv'.format(name), encoding='utf-8', index=False)
    round(df.describe(),2).to_csv('{}_described.csv'.format(name))


# Twitter: actualizando dataset historico

In [ ]:
#Twitter
df_new = pd.read_csv('Twitter_Mayo_clean.csv', encoding='utf-8')
df_old = pd.read_csv('historical_twitter_raw_clean.csv', encoding='utf-8')
df_update = pd.concat([df_old, df_new], sort=False)
df_update.to_csv('twitter historical_updated 05 2021.csv', index=False, encoding='utf-8',)

# Facebook: Actualizando dataset historico

In [ ]:
#Facebook
df_nuevo = pd.read_csv('Fb_Contenido_clean.csv', encoding='utf-8')
df_viejo = pd.read_csv('FB_historical_clean.csv', encoding='utf-8')
df_actual = pd.concat([df_viejo,df_nuevo], sort=False)
df_actual.to_csv('fb_historical_updated 05 2021.csv', index=False, encoding='utf-8')

# TikTok: Actualizando dataset historico

In [ ]:
#TikTok
df_mes = pd.read_csv('TikTok_Mayo_clean.csv')
df_hist = pd.read_csv('tiktok_overall_clean.csv')
df_act = pd.concat([df_hist,df_mes], sort=False)
df_act.to_csv('tiktok_historical_updated 05 2021.csv', index=False, encoding='utf-8')

# Concatenando Twitter data historica

In [ ]:

#os.listdir('.')
months = [
 'twitter sep 2020.csv',
 'twitter feb 2021.csv',
 'twitter ene 2021.csv',
 'twitter dic 2020.csv',
 'twitter oct 2020.csv',
 'twittter nov 2020.csv',
 'twitter abr 2021.csv',
 'twitter mar 2021.csv']

df_ene = pd.read_csv('twitter ene 2021.csv', encoding='utf-8')
df_dic = pd.read_csv('twitter dic 2020.csv', encoding='utf-8')
df_nov = pd.read_csv('twittter nov 2020.csv', encoding='utf-8')
df_sep = pd.read_csv('twitter sep 2020.csv', encoding='utf-8')
df_oct = pd.read_csv('twitter oct 2020.csv', encoding='utf-8')
df_feb = pd.read_csv('twitter feb 2021.csv', encoding='utf-8')
df_mar = pd.read_csv('twitter mar 2021.csv', encoding='utf-8')
df_abr = pd.read_csv('twitter abr 2021.csv', encoding='utf-8')


historical_twitter = pd.concat([df_sep,
                                 df_oct,
                                 df_nov,
                                 df_dic,
                                 df_ene,
                                 df_feb,
                                 df_mar,
                                 df_abr], sort=False)
historical_twitter.to_csv('historical_twitter_raw.csv', index=False, encoding='utf-8')
twitter_cleaner('historical_twitter_raw')

"""
#historical_twitter.dropna(axis=0,how='any', subset=['Hora de publicación'], inplace=True)
#removing NAs

#fb_cleaner_contenido('FB_Sep2020_Abr2021_RAW')
#df_historic = pd.read_csv('FB_Sep2020_Abr2021_RAW_clean.csv',encoding='utf-8')
#sep_abr_2021 = pd.concat([df_historic, df_abr], sort=False)
#sep_abr_2021.to_csv('FB_historical.csv',encoding='utf-8', index=False)
#fb_cleaner_contenido('FB_historical')
"""

KeyError: ignored

# Concatenando Facebook data historica

In [ ]:
os.listdir('.')
months = [ 'Facebook_Mar_2021.csv',
 'Facebook_Dic_2020.csv',
 'Facebook_Oct_2020.csv',
 'Facebook_Abril_2021.csv',
 'Facebook_Sep_2020.csv',
 'Facebook_Feb_2021.csv',
 'Facebook_Nov_2020.csv']

df_ene = pd.read_csv('Facebook_Ene_2021.csv', encoding='utf-8')
df_dic = pd.read_csv('Facebook_Dic_2020.csv', encoding='utf-8')
df_nov = pd.read_csv('Facebook_Nov_2020.csv', encoding='utf-8')
df_sep = pd.read_csv('Facebook_Sep_2020.csv', encoding='utf-8')
df_oct = pd.read_csv('Facebook_Oct_2020.csv', encoding='latin-1')
df_feb = pd.read_csv('Facebook_Feb_2021.csv', encoding='latin-1')
df_mar = pd.read_csv('Facebook_Mar_2021.csv', encoding='utf-8')
df_abr = pd.read_csv('Facebook_Abril_2021.csv', encoding='utf-8')


historical_facebook = pd.concat([df_ene,
df_dic,
df_nov,
df_sep,
df_oct,
df_feb,
df_mar], sort=False)
historical_facebook.dropna(axis=0,how='any', subset=['Hora de publicación'], inplace=True)
#removing NAs

fb_cleaner_contenido('FB_Sep2020_Abr2021_RAW')
df_historic = pd.read_csv('FB_Sep2020_Abr2021_RAW_clean.csv',encoding='utf-8')
sep_abr_2021 = pd.concat([df_historic, df_abr], sort=False)
sep_abr_2021.to_csv('FB_historical.csv',encoding='utf-8', index=False)
fb_cleaner_contenido('FB_historical')
#historical_facebook.to_csv('historical_facebook_sep2020_abr2021.csv', index=False, encoding='utf-8')

FileNotFoundError: ignored

## Experimentacion 

In [ ]:
#fb_cleaner_contenido('Contenido')
#twitter_cleaner('Twitter')
#fb_cleaner_alcance('Tendencias')
#tiktok_overall('tiktok_overall')

In [ ]:
fb_cleaner_contenido('Fb_Contenido')
twitter_cleaner('Twitter_Mayo')
fb_cleaner_alcance('Fb_Tendencias')
tiktok_overall('TikTok_Mayo')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Afiliaciones

In [ ]:
df = pd.ExcelFile('afiliados.xlsx')
df.sheet_names
df = df.parse('Respuestas de formulario 1')
df = df.drop(columns=['Nombre Completo', 'Fecha de Nacimiento', 'Ocupacion',
       'Teléfono', 'Correo de Contacto', 'Consentimiento', 'Numero de INE', 'Consentimiento.1','Unnamed: 11',
       'Newsletter 1 - Send Status' ], axis=1)
df.dropna(axis=0, how='all', inplace=True)

In [ ]:
df['Estado al que pertenece tu municipio'] = df['Estado al que pertenece tu municipio'].apply(lambda x: x.strip())
df.sort_values(by='Estado al que pertenece tu municipio', ascending=True, inplace=True)

In [ ]:
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 721 entries, 502 to 32
Data columns (total 3 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Marca temporal                        721 non-null    datetime64[ns]
 1   Municipio en el que vives             721 non-null    object        
 2   Estado al que pertenece tu municipio  721 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 22.5+ KB


In [ ]:
df['Estado al que pertenece tu municipio'].value_counts()

Ciudad de México         63
CDMX                     54
Estado de México         50
Jalisco                  44
Baja California          43
Sonora                   29
Puebla                   27
Guanajuato               26
Nuevo León               25
Cdmx                     23
Querétaro                19
Veracruz                 19
México                   17
Tamaulipas               17
Coahuila                 16
Hidalgo                  16
San Luis Potosí          13
Chihuahua                13
Yucatán                  12
Michoacán                12
Quintana Roo             11
Estado de Mexico          9
Sinaloa                   9
Morelos                   7
Nuevo Leon                6
Zacatecas                 6
Chiapas                   6
Ciudad de mexico          6
Durango                   6
Mexico                    6
Queretaro                 5
Michoacan                 5
Oaxaca                    5
Campeche                  5
Ciudad de Mexico          4
Aguascalientes      

In [ ]:
Ciudad_de_mexico = ['CDMX', 
                    'cdmx',
                    'CdMx',
                    'cdmx',
                    'Cdmx',
                    'Ciudad de mexico',
                    'CDMX - Benito Juárez',
                    'Ciudad de México - Benito Juárez',
                    'Cd de México',
                    'Coudad De Mexico',
                    'Cudad de México',
                    'CD de Mexico',
                    'Ciudad de Mexico',
                    'CIUDAD DE MÉXICO',
                    'ciudad de mexico',
                    'Álvaro Obregón',
                    'Ciudad de Mexico',
                    'CDMX - Benito Juárez',
                    'Ciudad De México',
                    'Benito Juárez',
                    'De México'
                    ]


In [ ]:
for i in Ciudad_de_mexico:
    df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace(i,'Ciudad de México'))

In [ ]:
baja_cali = [
            'BAJA CALIFORNIA'     ,
            'Baja California Norte',

            'Baja Califronia'      ,
            'Baja california'      ,
            'baja california']

for i in baja_cali:
    df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace(i,'Baja California'))

baja_cali_2= [
            'Baja Baja California' ,         
            'Baja Baja California' ,]
for i in baja_cali:
    df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace(i,'Baja California'))

df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Baja Californiaa','Baja California'))


edo_mex = [
           'Estado Ciudad de México' ,
            'Estado De Mexico' ,
            'Estado De México' ,              
            'Estado de México.' ,              
            'Estado de mexico',
            'estado de mexico' ]

for i in edo_mex:
    df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace(i,'Estado de México'))

In [ ]:
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('JALISCO','Jalisco'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Estado de Mexico','Estado de México'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Queretaro','Querétaro'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('QUERETARO','Querétaro'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('QUERÉTARO','Querétaro'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('S.L.P.','San Luis Potosí'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('S.L.P','San Luis Potosí'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('SLP','San Luis Potosí'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('CHIAPAS','Chiapas'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(Californi)(\W|$)', 'Baja California', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Coahuila de Zaragoza','Coahuila'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(EDOMEX|Edomex)(\W|$)', 'Estado De México', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Edo. México','Estado De México'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Edo. de Mex.','Estado De México'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Estado de Oaxaca','Oaxaca'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(GUERRERO)(\W|$)', 'Guerrero', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(HIDALGO)(\W|$)', 'Hidalgo', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(Mexic| Mexico| México| México\.)(\W|$)', 'Ciudad de México', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Michoacán de Ocampo','Michoacán'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Michoacan','Michoacán'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('michoacan','Michoacán'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('yucatan','Yucatán'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Yucatan','Yucatán'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Tamaulips','Tamaulipas'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('TABASCO','Tabasco'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('San luis Potosí','San Luis Potosí'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('SONORA','Sonora'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(N\,L\.)(\W|$)', 'Nuevo León', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Nuevo Leon','Nuevo León'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(PUEBLA)(\W|$)', 'Puebla', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Quintana roo','Quintana Roo'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('QUINTANA ROO','Quintana Roo'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(\W|^)(Zacatecas\.)(\W|$)', 'Zacatecas', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(^)(México\.)', 'Estado de México', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: re.sub('(^)(México)', 'Estado de México', x))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('SonoraCiudad de México','Ciudad de México'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Estado De México','Estado de México'))
df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace('Mexico','Estado de México'))

baja_sur = ['Baja Baja Californi sur',
            'Baja california sur',
            'Baja California sur',]
for i in baja_sur:
    df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: x.replace(i,'Baja California Sur'))



adios = ['IL','itapua','Tarija','Guayaquil- Ecuador','Octava región','Ecuador']
def bye(x):
    if x in adios:
        x = np.nan
    return x

for i in adios:
    df['Estado al que pertenece tu municipio']=df['Estado al que pertenece tu municipio'].apply(lambda x: bye(x))

df.dropna(axis=0, how='any', inplace=True)



In [ ]:
df.to_csv('afiliados_v4.csv', encoding='utf-8')

In [ ]:
sum(df['Estado al que pertenece tu municipio'].value_counts())

715

In [ ]:
df.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 721 entries, 502 to 32
Data columns (total 3 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Marca temporal                        721 non-null    datetime64[ns]
 1   Municipio en el que vives             721 non-null    object        
 2   Estado al que pertenece tu municipio  715 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 42.5+ KB


In [ ]:
df_estados = df['Estado al que pertenece tu municipio'].value_counts()
df_municipios = df['Municipio en el que vives'].value_counts()

In [ ]:
df_temp = pd.read_csv('afiliados_v4.csv', encoding='utf-8')
df_temp['Fecha'] = df_temp['Marca temporal']
df_temp['Fecha'] = df_temp['Fecha'].apply(lambda x: x[0:10])

In [ ]:
df_temp.sample(10)

,Unnamed: 0,Marca temporal,Municipio en el que vives,Estado al que pertenece tu municipio,Fecha
643,577,2021-03-25 10:20:52.284,CENTRO,Tabasco,2021-03-25
42,20,2020-09-07 01:37:11.138,Ensenada,Baja California,2020-09-07
543,328,2020-12-08 09:19:43.625,Puebla,Puebla,2020-12-08
172,339,2020-12-08 22:28:18.805,Tlalpan,Ciudad de México,2020-12-08
638,710,2021-04-30 09:18:48.772,Guaymas,Sonora,2021-04-30
522,106,2020-09-12 21:01:30.101,San Andrés Cholula,Puebla,2020-09-12
89,16,2020-09-07 00:06:18.919,Cuauhtemoc,Ciudad de México,2020-09-07
464,207,2020-10-15 01:25:00.771,Naucalpan,Estado de México,2020-10-15
462,529,2021-03-11 10:42:05.502,Tlalnepantla,Estado de México,2021-03-11
77,361,2020-12-10 23:01:10.468,Iztapalapa,Ciudad de México,2020-12-10


In [ ]:
df_temp.to_csv('afiliados_v5.csv', encoding='utf-8')